# 1

In [5]:
import math

def epsilon_exact(q, m):
    if q <= 1:
        return 0.0

    prod = 1.0
    for i in range(1, q):
        prod *= 1.0 - i / m
    return 1.0 - prod


def epsilon_estimate(q, m):
    if q <= 1:
        return 0.0

    exponent = -(q * (q - 1)) / (2.0 * m)
    return 1.0 - math.exp(exponent)


def relative_error(e_exact, e_est):
    if e_exact == 0.0:
        return 0.0
    return abs(e_exact - e_est) / e_exact

def print_table(data):
    print(f"{'Q':>2} | {'theorem':>9} | {'estimate':>9} | {'error':>9}")
    print("----------------------------------------")
    for q, e_ex, e_es, err in data:
        print(f"{q:2d} | {e_ex:9.6f} | {e_es:9.6f} | {err:9.6f}")
    print("----------------------------------------")


q_min = 15
q_max = 30
m = 365
data = []
for q in range(q_min, q_max + 1):
    e_ex = epsilon_exact(q, m)
    e_es = epsilon_estimate(q, m)
    err = relative_error(e_ex, e_es)
    data.append((q, e_ex, e_es, err))

print_table(data)

 Q |   theorem |  estimate |     error
----------------------------------------
15 |  0.252901 |  0.249992 |  0.011504
16 |  0.283604 |  0.280189 |  0.012040
17 |  0.315008 |  0.311061 |  0.012528
18 |  0.346911 |  0.342413 |  0.012967
19 |  0.379119 |  0.374055 |  0.013355
20 |  0.411438 |  0.405805 |  0.013692
21 |  0.443688 |  0.437488 |  0.013975
22 |  0.475695 |  0.468938 |  0.014205
23 |  0.507297 |  0.500002 |  0.014381
24 |  0.538344 |  0.530536 |  0.014504
25 |  0.568700 |  0.560412 |  0.014573
26 |  0.598241 |  0.589513 |  0.014589
27 |  0.626859 |  0.617736 |  0.014554
28 |  0.654461 |  0.644993 |  0.014468
29 |  0.680969 |  0.671208 |  0.014334
30 |  0.706316 |  0.696320 |  0.014153
----------------------------------------


# 2

In [6]:
def find_q_for(p, m):
    lo, hi = 1, m + 1  # hi is exclusive
    while lo < hi:
        mid = (lo + hi) // 2
        eps = epsilon_exact(mid, m)
        if eps >= p:
            hi = mid
        else:
            lo = mid + 1
    q = lo
    return q, epsilon_exact(q, m), epsilon_estimate(q, m)

targets = [0.5, 0.75, 0.99999]
m = 6903361
for p in targets:
    q, exact, est = find_q_for(p, m)
    print(f"p = {p:6.5%} → Q = {q:3d}  (theorem = {exact:.6f}, estimate = {est:.6f})")

p = 50.00000% → Q = 3094  (theorem = 0.500039, estimate = 0.499987)
p = 75.00000% → Q = 4375  (theorem = 0.750002, estimate = 0.749929)
p = 99.99900% → Q = 12605  (theorem = 0.999990, estimate = 0.999990)


# 2.EC
6903361 is a balanced semiprime, it is the product of two close primes, which makes it trivially factorable with Fermat’s method. 

RSA encryption relies entirely on the mathematical asymmetry of factoring. A system generates two massive, secret prime numbers ($p$ and $q$) and multiplies them together to create a public modulus ($N$). The entire security model assumes that if $N$ is sufficiently large (like 2048 bits), it will take an attacker thousands of years of trial division to find the original $p$ and $q$

# 3
- The team first scanned ~100000 SSL/TLS server certificates and discovered that a handful of public Certification Authorities (CAs) were still signing end-entity certificates with MD5.  
 
- For a chosen-prefix collision an attacker must know every byte of the "to-be-signed" (TBS) certificate up to the point where the collision blocks start. Two fields are normally filled in by the CA, not the requester:  
    - Validity period ("Not Before/Not After")  
    - Serial number  
- The researchers therefore had to predict both:  
   - Validity: By observing that RapidSSL’s system always issued a certificate exactly six seconds after the purchaser clicked "OK", they could pick the submission time so that the CA’s clock would place any desired start date/time in the TBS.  
   - Serial number: Analysis of 9251 RapidSSL certs showed the numbers were simply incremented. Over a weekend the CA issued ~800-1000 certs. By buying a few dozen cheap certificates they could "steer" the counter so that, at a pre-selected second T, the CA would issue the predicted serial S.

- They prepared two different TBS certificate templates (prefixes):  
    - "Real" certificate - a normal end-entity SSL certificate request that RapidSSL would happily sign.  
    - "Rogue CA" certificate - an intermediate CA certificate for which the attackers controlled the private key and whose BasicConstraints extension was set to `CA = TRUE`.

- In order to acheive two different overall messages $M$ and $M'$ to hash to the same value you can try to construct them as
$M  =  P  \|  C  \|  S$
$M' =  P' \|  C' \| S$
- The two prefixes $P$ and $P'$ differed in many fields (subject DN, public key, BasicConstraints, etc.). Prefix $P =$ first 474 bytes of the legitimate end-entity certificate, up to the start of its RSA modulus. Prefix $P' =$ first 477 bytes of their specially crafted rogue CA certificate, up to the “tumor” extension that precedes its own public key and BasicConstraints. Suffix $S =$ all the remaining bytes that must be identical in both certs. They then ran Marc Stevens' algorithm to generate the pair collision blocks $(C,C')$ that forces $MD5(P\|C\|S)$ to equal $MD5(P'\|C'\|S)$. 

- They now submitted the real CSR to RapidSSL at the pre-arranged second T. RapidSSL generated the predicted serial number and validity period, appended its issuer data, ran MD5 over the TBS portion, and signed the resulting hash with the Equifax root key. Because the rogue TBS had the same MD5 hash, the very same signature verified it as well.

## Certificate similarities and differences
Similarities between certs:
- Issuer Distinguished Name (the real RapidSSL/Equifax root).  
- Serial number (attacker set the rogue cert to the predicted value the CA would assign).  
- MD5 hash of the TBS section and therefore the RSA signature.

Differences between the certs:
- Subject DN - the real cert used a deliberately long common name, the rogue CA a very short one to make room for fields before the collision block.  
- Public-key modulus - 2048-bit key in the real cert (collision hidden inside), 1024-bit key in the rogue CA (so attackers could use it in practice).  
- BasicConstraints - `CA = FALSE` in the real cert, `CA = TRUE` (no pathLenConstraint) in the rogue one, turning it into an intermediate CA certificate.  
- Validity - the real cert had the normal one-year validity that RapidSSL would issue; the rogue CA certificate was deliberately back-dated to August 2004 so it could not be abused later.  

## Conclusions

Any browser that trusted the legitimate Equifax / RapidSSL root would also accept any leaf certificate signed by the attackers’ rogue CA. This meant practical, almost undetectable man-in-the-middle attacks on HTTPS, e-mail TLS, code signing, etc. 

Industry response:  
- Browsers (Mozilla, Microsoft, Apple, etc.) and OS vendors quickly blocked further acceptance of MD5-signed certificates and began flagging them as insecure.  
- The CA/Browser Forum updated its Baseline Requirements: since 2009 commercial CAs have been forbidden to issue new certificates signed with MD5; later the same happened to SHA-1.  
- CAs were urged to stop using MD5, to set pathLenConstraints when issuing end-entity certificates only, and to audit serial-number generation to prevent predictability.  

The attack conclusively proved that the theoretical collision vulnerabilities of MD5 can be weaponised against real-world PKI systems, accelerating the migration to stronger hash functions (SHA-2 family) throughout TLS ecosystems.

In short, by exploiting MD5’s chosen-prefix collision weakness and some operational shortcomings at a CA, the researchers produced two distinct X.509 certificates with identical MD5 hashes and signatures, a normal one and a fully-privileged intermediate CA under their control. The demonstration forced the immediate deprecation of MD5 in certificate signing and highlighted the need for robust CA operational practices and stronger cryptographic primitives in TLS.